In [15]:
from sqlalchemy import create_engine
import sqlite3
import pandas as pd
import pickle

# Load your pickle file
with open("/Users/evro/Documents/code/python/fetch/data/validated/users.pkl", "rb") as f:
    users = pickle.load(f)
with open("/Users/evro/Documents/code/python/fetch/data/validated/receipts.pkl", "rb") as f:
    receipts = pickle.load(f)
with open("/Users/evro/Documents/code/python/fetch/data/validated/receipt_item_data.pkl", "rb") as f:
    receipt_items = pickle.load(f)
with open("/Users/evro/Documents/code/python/fetch/data/validated/brands.pkl", "rb") as f:
    brands = pickle.load(f)

# Create in-memory SQLite DB and load data into separate tables
conn = sqlite3.connect(':memory:')
users.to_sql('users', conn, index=False, if_exists='replace')
receipts.to_sql('receipts', conn, index=False, if_exists='replace')
receipt_items.to_sql('receipt_items', conn, index=False, if_exists='replace')
brands.to_sql('brands', conn, index=False, if_exists='replace')


# Create a SQLAlchemy engine tied to the same connection
from sqlalchemy import create_engine
engine = create_engine('sqlite://', creator=lambda: conn)

# Run a query using pandas
query = """
WITH recent_month AS (
  SELECT
    date(strftime('%Y-%m-01', 'now'), '-1 month') AS start_date,
    strftime('%Y-%m-01', 'now') AS end_date
)
SELECT
  b.name AS brand_name,
  COUNT(DISTINCT r.receipt_id) AS receipt_count
FROM receipts r
JOIN receipt_items ri ON r.receipt_id = ri.receipt_id
JOIN brands b ON ri.brand_code = b.brand_code
JOIN recent_month rm
  ON r.date_scanned >= rm.start_date
 AND r.date_scanned < rm.end_date
GROUP BY b.name
ORDER BY receipt_count DESC
LIMIT 5;
"""

result = pd.read_sql_query(query, engine)
result


,brand_name,receipt_count
